In [1]:
import numpy as np
import cv2

import PIL.Image as Image
import os

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.models import Sequential

import pathlib

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.layers import Concatenate


import warnings
warnings.filterwarnings("ignore")

In [2]:

IMG_SIZE = (224,224)
IMG_SHAPE = IMG_SIZE + (3,)


data_dir = pathlib.Path('./Image data')
images=list(data_dir.glob('*/*.tif'))

image_dic= {
    'normal' : list(data_dir.glob("e0_Normal/*")),
    'hole':list(data_dir.glob("e1_hole/*")),
    'stain' : list(data_dir.glob("e2_Stain/*")),
    'net' :list(data_dir.glob("e3_Net/*")),
    'color' :list(data_dir.glob("e4_colour/*")),
    'crease': list(data_dir.glob("e5_Crease/*")),
}



labels = {
    'normal':0,
    'hole':1,
    'stain':2,
    'net':3,
    'color':4,
    'crease':5,
}



X,y=[],[]

for name, images in image_dic.items():
    for image in images:
        img = cv2.imread(str(image))
        
        
        if img is None:
            print('Wrong path:', image)
        else:
            resized = cv2.resize(img,IMG_SIZE)
            X.append(resized)
            y.append(labels[name])



X = np.array(X)
y= np.array(y)
X_train, X_test, y_train,y_test = train_test_split(X,y,random_state=0, shuffle=True,test_size=0.2)
X_train = X_train/255
X_test = X_test/255

Wrong path: Image data/e0_Normal/.DS_Store


In [3]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
# base_model = tf.keras.applications.mobilenet.MobileNet(input_shape=(224,224,3),
#                                                include_top=False,
#                                                weights='imagenet')


In [4]:
def ensemble(x, inputs, classes):
    seps  =[]
    for i in range(classes):
        res=tf.keras.layers.GlobalAveragePooling2D()(x)
        res = tf.keras.layers.Dense(128, activation='relu')(res)
        res = tf.keras.layers.Dropout(0.4)(res)
        res = tf.keras.layers.Dense(96, activation='relu')(res)
        #res= tf.keras.layers.Dense(60,activation='relu',name='predictions'+str(i))(res)
        #res = tf.keras.layers.Dense(64, activation='softmax')(res)
        res = tf.keras.layers.Dense(32, activation='tanh')(res)

        seps.append(res)
    out= Concatenate()(seps)
    out = tf.keras.layers.Dense(6, activation='softmax')(out)
    model = tf.keras.Model(inputs,out)
    return model
input_img = tf.keras.Input(IMG_SHAPE, name='input')
model1= tf.keras.applications.mobilenet.MobileNet(input_tensor = input_img, include_top= False)
model_mn = tf.keras.Sequential()

for layers in model1.layers[:50]:
    model_mn.add(layers)

inputs = tf.keras.Input(shape=(224, 224, 3))
x= model_mn(inputs)
main_model= ensemble(x,inputs,6)

for layers in main_model.layers[45:50]:
    layers.trainable= False

main_model.compile(optimizer = Adam(learning_rate=0.0005), loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['categorical_accuracy'])

In [5]:
main_model.fit(X_train,to_categorical(y_train) ,epochs=60,
                            validation_data=(X_test, to_categorical(y_test)))

Epoch 1/60


2023-04-05 11:07:52.753834: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


31/31 [==============================] - 43s 1s/step - loss: 2.0292 - categorical_accuracy: 0.1653 - val_loss: 1.9671 - val_categorical_accuracy: 0.1577
Epoch 2/60
31/31 [==============================] - 38s 1s/step - loss: 1.8991 - categorical_accuracy: 0.1913 - val_loss: 1.8177 - val_categorical_accuracy: 0.1826
Epoch 3/60
31/31 [==============================] - 38s 1s/step - loss: 1.8033 - categorical_accuracy: 0.2245 - val_loss: 1.7623 - val_categorical_accuracy: 0.2324
Epoch 4/60
31/31 [==============================] - 45s 1s/step - loss: 1.7697 - categorical_accuracy: 0.2672 - val_loss: 1.8462 - val_categorical_accuracy: 0.1950
Epoch 5/60
31/31 [==============================] - 40s 1s/step - loss: 1.6326 - categorical_accuracy: 0.3202 - val_loss: 2.1887 - val_categorical_accuracy: 0.2407
Epoch 6/60
31/31 [==============================] - 40s 1s/step - loss: 1.4812 - categorical_accuracy: 0.3836 - val_loss: 2.1170 - val_categorical_accuracy: 0.2531
Epoch 7/60
31/31 [=========

In [ ]:
main_model.evaluate(X_test, to_categorical(y_test))

8/8 [==============================] - 2s 235ms/step - loss: 0.2697 - categorical_accuracy: 0.9253


[0.26972249150276184, 0.9253112077713013]

In [12]:
main_model.save("ensemble_9544_mobilenetv1")

INFO:tensorflow:Assets written to: ensemble_9544_mobilenetv1/assets


INFO:tensorflow:Assets written to: ensemble_9544_mobilenetv1/assets
